# 🌍 V2 Location Features Engineering

Add enhanced geographic features for location-based prediction:
- Hemisphere (Northern/Southern)
- Climate Zone (Tropical/Subtropical/Temperate/Continental/Polar)
- Latitude Band
- Season adjustment for hemisphere

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

print("✅ Libraries loaded")

✅ Libraries loaded


## 1. Load Original Data

In [2]:
# Load V1 country stats
stats = pd.read_csv('../../models/country_stats.csv')
print(f"📊 Loaded {len(stats)} countries")
stats.head()

📊 Loaded 186 countries


,country,latitude,longitude,temp_mean,temp_std,temp_min,temp_max,country_encoded,temp_mean_month_1,temp_mean_month_2,temp_mean_month_3,temp_mean_month_4,temp_mean_month_5,temp_mean_month_6,temp_mean_month_7,temp_mean_month_8,temp_mean_month_9,temp_mean_month_10,temp_mean_month_11,temp_mean_month_12
0,Afghanistan,34.517341,69.182659,20.829496,10.244962,-3.3,36.3,0,1.312903,4.789286,11.993548,22.986667,27.267742,29.747826,32.118333,30.664516,27.525000,21.125806,14.796667,8.252727
1,Albania,41.327986,19.819114,21.133453,8.807284,1.3,39.2,1,13.425806,12.428571,16.067742,18.403333,21.664516,30.123913,32.428333,31.601613,25.445000,18.503226,13.103333,9.334545
2,Algeria,36.762502,3.050484,21.964324,6.912636,2.1,38.2,2,15.019355,15.425000,17.616129,19.700000,22.274194,28.360870,29.794915,29.598387,26.336667,21.398387,16.051667,12.681818
3,Andorra,42.500000,1.517341,10.941007,9.101585,-13.0,28.9,3,2.035484,4.128571,1.180645,7.210000,11.038710,20.506522,21.461667,22.104839,14.220000,10.474194,2.888333,0.581818
4,Angola,-8.838630,13.233545,26.044964,2.755828,19.3,32.2,4,29.380645,29.264286,29.548387,28.626667,27.861290,25.628261,23.348333,22.738710,24.326667,25.432258,26.718333,26.967273


## 2. Add Hemisphere Feature

In [3]:
# Hemisphere based on latitude
stats['hemisphere'] = np.where(stats['latitude'] >= 0, 'Northern', 'Southern')
stats['hemisphere_encoded'] = np.where(stats['latitude'] >= 0, 1, 0)

print("Hemisphere distribution:")
print(stats['hemisphere'].value_counts())

Hemisphere distribution:
hemisphere
Northern    146
Southern     40
Name: count, dtype: int64


## 3. Add Climate Zone Feature

In [4]:
def get_climate_zone(lat):
    """Classify climate zone based on absolute latitude."""
    abs_lat = abs(lat)
    if abs_lat <= 23.5:
        return 'Tropical'
    elif abs_lat <= 35:
        return 'Subtropical'
    elif abs_lat <= 55:
        return 'Temperate'
    elif abs_lat <= 66.5:
        return 'Continental'
    else:
        return 'Polar'

stats['climate_zone'] = stats['latitude'].apply(get_climate_zone)

# Encode climate zones
zone_mapping = {'Tropical': 0, 'Subtropical': 1, 'Temperate': 2, 'Continental': 3, 'Polar': 4}
stats['climate_zone_encoded'] = stats['climate_zone'].map(zone_mapping)

print("\nClimate Zone distribution:")
print(stats['climate_zone'].value_counts())


Climate Zone distribution:
climate_zone
Tropical       93
Temperate      57
Subtropical    28
Continental     8
Name: count, dtype: int64


In [5]:
# Visualize on world map
fig = px.scatter_geo(stats, lat='latitude', lon='longitude', color='climate_zone',
                     hover_name='country', title='🌍 Countries by Climate Zone',
                     color_discrete_sequence=['#ff6b6b', '#feca57', '#48dbfb', '#1dd1a1', '#5f27cd'])
fig.update_geos(showland=True, landcolor='#1a1a2e', oceancolor='#0f0f1a',
                showocean=True, bgcolor='#0f0f1a')
fig.update_layout(paper_bgcolor='#0f0f1a', font_color='white')
fig.show()

## 4. Add Latitude Band Feature

In [6]:
# Normalized absolute latitude (0-1 scale)
stats['abs_latitude'] = abs(stats['latitude'])
stats['latitude_normalized'] = stats['abs_latitude'] / 90.0

# Latitude band (10-degree increments)
stats['latitude_band'] = (stats['abs_latitude'] // 10).astype(int)

print("\nLatitude Band distribution:")
print(stats['latitude_band'].value_counts().sort_index())


Latitude Band distribution:
latitude_band
0    42
1    44
2    23
3    27
4    31
5    17
6     2
Name: count, dtype: int64


## 5. Temperature Patterns by Climate Zone

In [7]:
# Average temperature by climate zone
zone_temps = stats.groupby('climate_zone')['temp_mean'].agg(['mean', 'std', 'min', 'max']).round(1)
zone_temps = zone_temps.reindex(['Tropical', 'Subtropical', 'Temperate', 'Continental', 'Polar'])
print("\nTemperature by Climate Zone:")
zone_temps


Temperature by Climate Zone:


,mean,std,min,max
climate_zone,,,,
Tropical,25.5,3.7,10.1,32.2
Subtropical,24.8,6.5,10.3,33.6
Temperate,16.5,4.6,5.4,26.1
Continental,10.3,1.9,6.3,12.2
Polar,NaN,NaN,NaN,NaN


In [8]:
# Box plot of temperature by climate zone
fig = px.box(stats, x='climate_zone', y='temp_mean', color='climate_zone',
             title='🌡️ Temperature Distribution by Climate Zone',
             category_orders={'climate_zone': ['Tropical', 'Subtropical', 'Temperate', 'Continental', 'Polar']})
fig.update_layout(paper_bgcolor='#0f0f1a', plot_bgcolor='#0f0f1a', font_color='white')
fig.show()

## 6. Save Enhanced Stats

In [9]:
# Select columns for V2
v2_columns = [
    'country', 'latitude', 'longitude', 'country_encoded',
    'temp_mean', 'temp_std', 'temp_min', 'temp_max',
    'hemisphere', 'hemisphere_encoded',
    'climate_zone', 'climate_zone_encoded',
    'abs_latitude', 'latitude_normalized', 'latitude_band'
] + [f'temp_mean_month_{m}' for m in range(1, 13)]

v2_stats = stats[v2_columns]

# Save to V2 models folder
v2_stats.to_csv('../models/location_stats.csv', index=False)
print(f"✅ Saved location_stats.csv with {len(v2_stats.columns)} columns")
print(f"📊 Columns: {v2_stats.columns.tolist()}")

✅ Saved location_stats.csv with 27 columns
📊 Columns: ['country', 'latitude', 'longitude', 'country_encoded', 'temp_mean', 'temp_std', 'temp_min', 'temp_max', 'hemisphere', 'hemisphere_encoded', 'climate_zone', 'climate_zone_encoded', 'abs_latitude', 'latitude_normalized', 'latitude_band', 'temp_mean_month_1', 'temp_mean_month_2', 'temp_mean_month_3', 'temp_mean_month_4', 'temp_mean_month_5', 'temp_mean_month_6', 'temp_mean_month_7', 'temp_mean_month_8', 'temp_mean_month_9', 'temp_mean_month_10', 'temp_mean_month_11', 'temp_mean_month_12']


In [10]:
# Save climate zone mapping as JSON
import json

climate_data = {
    'zone_mapping': zone_mapping,
    'zone_temps': zone_temps.to_dict(),
    'thresholds': {
        'tropical': 23.5,
        'subtropical': 35,
        'temperate': 55,
        'continental': 66.5,
        'polar': 90
    }
}

with open('../models/climate_zones.json', 'w') as f:
    json.dump(climate_data, f, indent=2)
print("✅ Saved climate_zones.json")

✅ Saved climate_zones.json


## 7. Summary

In [11]:
print("="*50)
print("📊 V2 Location Features Summary")
print("="*50)
print(f"\nCountries: {len(v2_stats)}")
print(f"\nNew Features Added:")
print(f"  • hemisphere (Northern/Southern)")
print(f"  • climate_zone (Tropical/Subtropical/Temperate/Continental/Polar)")
print(f"  • abs_latitude (absolute value)")
print(f"  • latitude_normalized (0-1 scale)")
print(f"  • latitude_band (10° increments)")
print(f"\nFiles Saved:")
print(f"  • v2/models/location_stats.csv")
print(f"  • v2/models/climate_zones.json")
print("\n✅ Ready for model training!")

📊 V2 Location Features Summary

Countries: 186

New Features Added:
  • hemisphere (Northern/Southern)
  • climate_zone (Tropical/Subtropical/Temperate/Continental/Polar)
  • abs_latitude (absolute value)
  • latitude_normalized (0-1 scale)
  • latitude_band (10° increments)

Files Saved:
  • v2/models/location_stats.csv
  • v2/models/climate_zones.json

✅ Ready for model training!
